In [ ]:
import numpy as np
from numba import cuda
# need to require cudatoolkit

@cuda.jit
def jfa_step(grid, seeds, step, n):
    x, y = cuda.grid(2)
    if x >= n or y >= n:
        return

    for dx in [-step, 0, step]:
        for dy in [-step, 0, step]:
            nx, ny = x + dx, y + dy
            if 0 <= nx < n and 0 <= ny < n:
                seed_x, seed_y = seeds[nx, ny]
                if seed_x != -1:  # Valid seed
                    dist = (x - seed_x) ** 2 + (y - seed_y) ** 2
                    current_dist = (x - grid[x, y, 0]) ** 2 + (y - grid[x, y, 1]) ** 2
                    if current_dist == 0 or dist < current_dist:
                        grid[x, y, 0] = seed_x
                        grid[x, y, 1] = seed_y

def jump_flood_gpu(grid_size, seed_points):
    n = grid_size
    grid = np.full((n, n, 2), -1, dtype=np.int32)
    seeds = np.full((n, n, 2), -1, dtype=np.int32)
    for x, y in seed_points:
        seeds[x, y] = [x, y]

    d_grid = cuda.to_device(grid)
    d_seeds = cuda.to_device(seeds)

    step = n // 2
    threads_per_block = (16, 16)
    blocks_per_grid = (n // threads_per_block[0] + 1, n // threads_per_block[1] + 1)

    while step >= 1:
        jfa_step[blocks_per_grid, threads_per_block](d_grid, d_seeds, step, n)
        cuda.synchronize()
        step //= 2

    return d_grid.copy_to_host()

# Example usage
seed_points = [(10, 10), (50, 50), (90, 90)]
result = jump_flood_gpu(128, seed_points)




In [ ]:
print(result)

In [ ]:
from numba import cuda
cuda.detect()

In [ ]:

import numpy as np
from numba import cuda, float32

# Define the grid size
grid_size = 128

# Define the kernel to perform jump flooding with color
@cuda.jit
def jfa_kernel(grid, seeds, step, n, color_grid):
    x, y = cuda.grid(2)

    if x >= n or y >= n:
        return

    # Initialize the distance and color from the seed points
    min_dist = float32(1e10)  # Default high distance
    min_color = (float32(0.0), float32(0.0), float32(0.0))  # Default color (black)

    # Loop through the seed points, considering the current step
    for i in range(len(seeds)):
        sx, sy = seeds[i]
        dist = abs(x - sx) + abs(y - sy)  # Manhattan distance
        # Only consider points within the "step" distance
        if dist <= step:
            if dist < min_dist:
                min_dist = dist
                min_color = (color_grid[sx, sy][0], color_grid[sx, sy][1], color_grid[sx, sy][2])

    # Assign the minimum distance and color found
    grid[x, y] = min_dist
    color_grid[x, y] = min_color

# Initialize the grid, color grid, and seeds
grid = np.zeros((grid_size, grid_size), dtype=np.float32)
# color_grid = np.zeros((grid_size, grid_size, 3), dtype=np.float32)  # RGB color grid
# color_grid.fill(np.random.rand(grid_size, grid_size, 3))
color_grid = np.random.rand(grid_size, grid_size, 3).astype(np.float32)
seeds = [(10, 10), (50, 50), (90, 90)]  # Example seed points

# Create a CUDA device array for the color grid and the grid
d_grid = cuda.to_device(grid)
d_color_grid = cuda.to_device(color_grid)
d_seeds = cuda.to_device(seeds)

# Perform Jump Flooding
blockdim = (16, 16)  # Threads per block
griddim = (grid_size // 16, grid_size // 16)  # Number of blocks

# Run the kernel for different steps
for step in [16, 8, 4, 2, 1]:
    jfa_kernel[griddim, blockdim](d_grid, d_seeds, step, grid_size, d_color_grid)

# Copy the result back to host
grid_result = d_grid.copy_to_host()
color_result = d_color_grid.copy_to_host()

# Visualize the color grid and distance grid using matplotlib
import matplotlib.pyplot as plt

# Visualize the color grid
plt.imshow(color_result)
plt.title("Color Grid Based on Jump Flooding")
plt.show()

# Visualize the distance grid
plt.imshow(grid_result, cmap='hot')
plt.title("Distance Grid")
plt.show()
